In [17]:
import pandas as pd
from openai import OpenAI
import glob
import os
from wandb.sdk.data_types.trace_tree import Trace
import wandb
import configparser
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import sys
import os
import inspect
import json
import random
from collections import Counter
import ast
# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config = configparser.ConfigParser()
# Read the configuration file
config.read('config.ini')
api_key_openai = config.get('credentials', 'api_key_openai')
api_key_mistral = config.get('credentials', 'api_key_mistral')
surfdrive_url_input_sentences = config.get('credentials', 'surfdrive_url_input_sentences')
surfdrive_url_prompts = config.get('credentials', 'surfdrive_url_prompts')
output_parallel_data = 'output_parallel_data/'
output_llm_folder_path = 'output_llm_data/'

### Create the context string

In [32]:
context_string = {}
output_chat_data = "output_chat_data/"
csv_files = glob.glob(output_chat_data + '*')
for file in csv_files:
    username = file[17:19]
    if (username=='U4'):
        if(file.find('word_distribution') != -1):
            df_words = pd.read_csv(file) 
            tuples_list = [(length, freq) for length, freq in zip(df_words['word_length'], df_words['percentage'])]
            word_count_distribution = str(tuples_list)
            print('Word frequency: ', output_string, '\n\n')
    
        elif(file.find('chat_llm') != -1):
            df_chat = pd.read_csv(file)
        
            adjectives_lists = [ast.literal_eval(keyword_str) for keyword_str in list(df_chat['adjectives'])]
            # Merge all lists into one
            all_adjectives = [keyword for sublist in adjectives_lists for keyword in sublist]
            adjectives_df = pd.DataFrame({'adjective': all_adjectives})
            # Count the frequency of each keyword
            adjectives_df = adjectives_df['adjective'].value_counts().reset_index()
            # Rename the columns
            adjectives_df.columns = ['adjective', 'frequency']
            total_keywords = adjectives_df['frequency'].sum()
            adjectives_df['percentage'] = round((adjectives_df['frequency'] / total_keywords) * 100,2)
            tuples_list = [(length, freq) for length, freq in zip(adjectives_df['adjective'], adjectives_df['percentage'])]
            adjective_distribution = str(tuples_list)

            print('Adjectives: ',adjective_distribution, '\n\n')
            # all_adjectives = []
            # for adjectives_list in list(df_chat['adjectives']):
            #     all_adjectives.extend(adjectives_list)
    
            # # Step 2: Count the frequency of each adjective
            # adjective_counts = Counter(all_adjectives)
    
            # # Step 3: Calculate the percentage of occurrence of each adjective
            # total_adjectives = len(all_adjectives)
            # adjective_percentages = {adj: count / total_adjectives * 100 for adj, count in adjective_counts.items()}
            # print(adjective_counts)
            # print(adjective_percentages)
            
        

Adjectives:  [('true', 7.78), ('good', 6.67), ('nice', 3.33), ('embarrassing', 2.22), ('best', 2.22), ('funny', 2.22), ('much', 2.22), ('national', 2.22), ('interesting', 2.22), ('fun', 2.22), ('short', 2.22), ('tiny', 2.22), ('low', 2.22), ('professional', 2.22), ('impossible', 1.11), ('boring', 1.11), ('covid', 1.11), ('worst', 1.11), ('conservative', 1.11), ('german', 1.11), ('cool', 1.11), ('German', 1.11), ('weird', 1.11), ('Bayern', 1.11), ('regular', 1.11), ('sooo', 1.11), ('stressful', 1.11), ('favorite', 1.11), ('relaxed', 1.11), ('insane', 1.11), ('True', 1.11), ('least', 1.11), ('late', 1.11), ('new', 1.11), ('enought', 1.11), ('non', 1.11), ('-', 1.11), ('fine', 1.11), ('long', 1.11), ('random', 1.11), ('little', 1.11), ('small', 1.11), ('Hard', 1.11), ('relaxing', 1.11), ('Welll', 1.11), ('same', 1.11), ('dense', 1.11), ('close', 1.11), ('Dutch', 1.11), ('fast', 1.11), ('LONG', 1.11), ('polite', 1.11), ('bad', 1.11), ('few', 1.11), ('artistic', 1.11), ('cheap', 1.11), ('si

In [23]:
keyword_counts

,adjective,frequency,percentage
0,true,7,7.777778
1,good,6,6.666667
2,nice,3,3.333333
3,embarrassing,2,2.222222
4,best,2,2.222222
...,...,...,...
61,awkward,1,1.111111
62,next,1,1.111111
63,19th,1,1.111111
64,serious,1,1.111111
